# Normal vs Leveraged Gains

In [ ]:
import time
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, Markdown, Latex

# darkmode = False

# If you have Jupyter Theme [https://github.com/dunovank/jupyter-themes]
# jt -t monokai -tfs 14 -ofs 11 -nfs 10 -dfs 12 -fs 11 -lineh 150
darkmode = True

## Gain Generator

The `gain_generator()` function produces a series of random daily gains. The default parameters produces a cummulative gain of 10% (`target = 1.1`) over 200 days (`count = 200`), with a daily standard deviation of 2% (`m = 0.02`). This closely resembles the SP-500. Users may also choose a uniform distribution by setting the keyword `normal` to `False`.

In [ ]:
def gain_generator_v1(target=1.1, count=200, m=0.02, tol=2e-5, verbose=0, normal=True):
    if normal:
        # Normally distributed numbers
        d = 1.0 + m * np.random.normal(size=count-1)
    else:
        # Uniform random numbers in between [-m, +m]
        d = 1.0 + 2.0 * m * np.random.random(count-1) - m
    # Find a gain that produces desired cumulative gain at the end of the period
    k = 0
    delta = 1
    offset = 0;
    mu = 1.0 / count
    while abs(delta) > tol and k < 20:
        x = d + offset
        ret = np.prod(x)
        delta = ret - target
        if verbose:
            print('{:3d}  o = {:.6f} -> {:.6f}  {:+9.6f}'.format(k, offset, ret, delta))
        offset -= mu * delta
        k += 1
    d = np.insert(d - 1 + offset, 0, 0)
    return d

In [ ]:
def gain_generator(target=1.1, count=200, m=0.02, tol=2e-5, verbose=0, normal=True):
    if isinstance(count, int):
        shape = count - 1
    elif isinstance(count, tuple) and len(count) == 2:
        shape = (count[0], count[1] - 1)
    else:
        print('Unable to generate for {}'.format(count))
        return None
    if normal:
        # Normally distributed numbers
        d = 1.0 + m * np.random.normal(size=shape)
    else:
        # Uniform random numbers in between [-m, +m]
        d = 1.0 + 2.0 * m * np.random.random(shape) - m
    # Find a gain that produces desired cumulative gain at the end of the period
    k = 0
    if isinstance(count, int):
        delta = 1
        offset = 0
        mu = 1.0 / count
        while abs(delta) > tol and k < 20:
            x = d + offset
            ret = np.prod(x)
            delta = ret - target
            if verbose:
                print('{:3d}  o = {:.6f} -> r = {:.6f}   d = {:+9.6f}'.format(k, offset, ret, delta))
            offset -= mu * delta
            k += 1
        d = np.insert(x - 1, 0, 0)
    else:
        shape = (count[0], 1)
        delta = np.ones(shape)
        offset = np.zeros(shape);
        mu = 1.0 / count[1]
        while np.any(abs(delta) > tol) and k < 20:
            x = d + offset
            ret = np.prod(x, axis=1, keepdims=True)
            delta = ret - target
            if verbose:
                print('{:3d}  o = {:.6f} -> r = {:.6f}   d = {:+9.6f}'.format(k, np.mean(offset), np.mean(ret), np.mean(delta)))
            offset -= mu * delta
            k += 1
        d = np.insert(x - 1, 0, 0, axis=1)
    return d

In [ ]:
# g = gain_generator(verbose=1)
# np.std(g)

In [ ]:
# g = gain_generator(count=(5, 10), verbose=1)

In [ ]:
fs = 0.5
if darkmode:
    edgecolor = 'white'
    facecolor = (1, 1, 1, 0.1)
    green_edge = '#99ff99'
    red_edge = '#ff9999'
    clr1 = '#4396f6'
    clr2 = '#bbff33'
    cmap = 'viridis'
else:
    edgecolor = 'black'
    facecolor = (0, 0, 0, 0.05)
    green_edge = '#004400'
    red_edge = '#440000'
    clr1 = '#4396f6'
    clr2 = '#33aa00'
    cmap = 'YlGnBu'

context_properties = {
    'font.family': 'sans-serif',
    'figure.frameon': False,
    'figure.facecolor': (0, 0, 0, 0),
    'axes.edgecolor': edgecolor,
    'axes.facecolor': facecolor,
    'axes.labelcolor': edgecolor,
    'axes.linewidth': 1.0 * fs,
    'axes.titleweight': 800,
    'axes.titlepad': 14.0,
    'axes.xmargin': 0,
    'axes.ymargin': 0,
    'grid.color': edgecolor,
    'hatch.color': edgecolor,
    'patch.edgecolor': edgecolor,
    'text.color': edgecolor,
    'xtick.color': edgecolor,
    'xtick.direction': 'in',
    'xtick.major.pad': 9.0 * fs,
    'xtick.major.size': 4.0 * fs,
    'xtick.major.width': 1.0 * fs,
    'ytick.color': edgecolor,
    'ytick.direction': 'in',
    'ytick.major.pad': 9.0 * fs,
    'grid.color': edgecolor,
    'grid.alpha': 0.1,
    'legend.frameon': False,
    'legend.framealpha': 0.15,
}
plt.rcParams['font.sans-serif'] = 'Arial'

## Simulation

This simulation emulates a random daily return with the targeted compound gain and a 3x leveraged return

In [ ]:
# Typical growth, low volatility derivative, e.g., XLU-UTSL, XLI-DUSL, XLP-NEED
# m = 0.003; gain = 1.08; count = 200

# Typical growth, moderate volatility, e.g., SPY-SPXL,
# m = 0.006; gain = 1.18; count = 400
# m = 0.006; gain = 1.09; count = 200

# High growth, high volatility, e.g., QQQ-TQQQ, XLK-TECL
# m = 0.006; gain = 1.16; count = 200
# m = 0.006; gain = 1.08; count = 100
# m = 0.006; gain = 1.04; count = 50

# Speculative, ultra-high volatility, e.g., FNGS-FNGU
# m = 0.017; gain = 1.090; count = 100
# m = 0.017; gain = 1.045; count = 50
# m = 0.017; gain = 1.0225; count = 25

# m = 0.017; count = 5; gain = 1 + 0.18 * count / 200
# m = 0.017; count = 10; gain = 1 + 0.18 * count / 200
# m = 0.017; count = 20; gain = 1 + 0.18 * count / 200
# m = 0.017; count = 100; gain = 1 + 0.18 * count / 200

# Say, it's going to be flat
m = 0.017; count = 50; gain = 1

# Simulation
g = gain_generator(gain, count=count+1, m=m)
t = np.arange(len(g))
n = np.prod(1.0 + g)
l = np.prod(1.0 + 3.0 * g)
cn = 100 * (np.cumprod(1 + g) - 1)
cl = 100 * (np.cumprod(1 + 3.0 * g) - 1)
lo = np.min(cl)
hi = np.max(cl)
hh = hi - lo
markersize = 8
delta = 0.5
tm, vm = np.argmax(cl), np.max(cl)

with plt.rc_context(context_properties):
    # Distribution of the Gains
    fig = plt.figure(dpi=110, figsize=(7.5, 8))
    fig.add_axes((0.05, 0.7, 0.9, 0.25))
    b = np.arange(-12 - 0.5 * delta, 12 + delta, delta)
    n, _, patches = plt.hist(300 * g, bins=b, alpha=0.6)
    for b, p in zip(b, patches):
        if b < -0.5 * delta:
            p.set_facecolor('red')
            p.set_edgecolor(red_edge)
        elif b < 0.5 * delta:
            p.set_facecolor('grey')
            p.set_edgecolor(edgecolor)
        else:
            p.set_facecolor('green')
            p.set_edgecolor(green_edge)
    plt.text(16, 0.93 * max(n), 'Up Days = {}\nDown Days = {}\nTotal Days = {}'.format(
        np.sum(g[1:] >= 0), np.sum(g[1:] < 0), count), linespacing=1.8, va='top', ha='right')
    plt.grid()
    plt.xlim((-17, 17))
    plt.xlabel('Gain (%)')
    plt.ylabel('Count')
    plt.title('Distribution of Daily Leveraged Gains ({:.1f}% bins)'.format(delta))

    # Raw Gain Values
    ax = plt.axes((0.05, 0.31, 0.9, 0.25))
    if len(t) <= 50:
        marker = '.'
    else:
        marker = ''
    ax.plot(t, 100.0 * g, marker=marker, markersize=markersize, color=clr1, label='Normal')
    ax.plot(t, 300.0 * g, marker=marker, markersize=markersize, color=clr2, label='Leveraged')
    ax.set_xlim((-0.03 * count, 1.12 * count))
    ax.set_ylim((-18, 18))
    ax.grid()
    ax.legend(loc='upper right', ncol=2)
    ax.set_xticklabels([''] * len(ax.get_xticklabels()))
    ax.set_ylabel('Daily Gain (%)')
    ax.set_title('Daily and Cummulative Gains During {} Days'.format(count))

    # Cumulative Gain Over the Period
    ax = fig.add_axes((0.05, 0.05, 0.9, 0.25))
    ax.plot(t, cn, label='Normal', color=clr1)
    ax.plot(t, cl, label='Leveraged', color=clr2)
    ax.plot(t[-1], cn[-1], marker='o', color=clr1)
    ax.text(t[-1] * 1.03, cn[-1], '{:.2f}%'.format(cn[-1]), color=clr1, va='center', fontweight=600)
    ax.plot(t[-1], cl[-1], marker='o', color=clr2)
    if cl[-1] > cn[-1]:
        y = max(cl[-1], cn[-1] + 0.15 * hh)
    else:
        y = min(cl[-1], cn[-1] - 0.15 * hh)
    ax.text(t[-1] * 1.03, y, '{:.2f}%'.format(cl[-1]), color=clr2, va='center', fontweight=600)
    if tm != t[-1]:
        ax.plot(tm, vm, marker='o', color=clr2)
        ax.text(tm, vm + 0.1 * hh, '{:.2f}%'.format(vm), color=clr2, fontweight=600)
    ax.set_xlim((-0.03 * count, 1.13 * count))
    ax.set_ylim(lo - 0.3 * hh, hi + 0.3 * hh)
    ax.grid()
    ax.set_xlabel('Days')
    ax.set_ylabel('Cummulative Gain (%)')

In [ ]:
plt.close()

## Multiple Realizations & Ensemble Heat Map

In [ ]:
# Multiple runs
r = 50000
gg = gain_generator(gain, (r, count + 1), m=m)
u = np.sum(gg >= 0, axis=1)                                         # Up Days
n = np.prod(1 + gg, axis=1)                                         # Normal gains
l = np.prod(1 + 3 * gg, axis=1)                                     # Leverage gains
h = np.cumprod(1 + 3 * gg, axis=1)                                  # Leverage gain history
n = 100 * (n - 1)                                                   # n in percentage
l = 100 * (l - 1)                                                   # l in percentage
h = 100 * (h - 1)                                                   # h in percentage
e = np.vstack((np.min(h, axis=1), np.max(h, axis=1))).transpose()   # Extremes
um = np.mean(u)                                                     # Average of up days
us = np.std(u)                                                      # STD of up days
nm = np.mean(n)                                                     # Mean normal return
lm = np.mean(l)                                                     # Mean leveraged return
ls = np.std(l)                                                      # STD of leveraged return
em = np.mean(e, axis=0)                                             # Mean extreme gains
es = np.std(e, axis=0)                                              # STD of extremes
ee = [np.min(e[:, 0]), np.max(e[:, 1])]                             # Extremas of extremems
hm = np.mean(h, axis=0)
hs = np.std(h, axis=0)

# Table
lab = {True: '+', False: '-'}
output  = '| \# | Up Days | Normal Return | 3x Leveraged | History | Profitable | Better? |\n'
output += '| --: | --: | :-: | :-: | :-: | :-: | :-: |\n'
def row(k):
    return ('| {} | {:3d} ({:.0f}%) | {:+.2f}% | {:+.2f}% '
            '| {:+.1f}% ... {:+.1f}% | {} | {} |\n').format(
                k + 1, u[k], 100 * u[k] / count, n[k], l[k],
                e[k, 0], e[k, 1], lab[e[k, 1] > 1], lab[l[k] > n[k]])
head, tail = 5, 3
for k in range(min(gg.shape[0] - tail, head)):
    output += row(k)
if gg.shape[0] > head:
    output += '| : | : | : | : | : | : | : |\n'
for k in range(gg.shape[0] - tail, gg.shape[0]):
    output += row(k)
output += '||\n'
output += ('| **Average** | **{:.1f} ({:.0f}%)** | **{:+.2f}%** | **{:+.2f}%** '
           '| **{:+.1f}% ... {:+.1f}%** | **{:.1f}%** | **{:.1f}%** |\n').format(
    um, 100 * um / count, nm, lm, em[0], em[1],
    100 * np.mean(e[:, 1] > 1), 100 * np.mean(l > n))
output += ('| **Standard** '
           '| {:.0f} ... {:.0f} | | {:+.2f}% ... {:+.2f}% | **{:+.1f}% ... {:+.1f}%** '
           '| |\n'.format(
    um - us, um + us, lm - ls, lm + ls, np.min(hm - hs), np.max(hm + hs)))
output += ('| **Extreme** '
           '| {} ... {} | | {:+.2f}% ... {:+.2f}% | {:+.1f}% ... {:+.1f}% '
           '| |\n'.format(
    np.min(u), np.max(u), np.min(l), np.max(l), ee[0], ee[1]))
display(Markdown(output))

# Heat map
phi = 5
showall = False
if showall:
    lo = np.floor(np.min(h) / phi) * phi
    hi = np.ceil(np.max(h) / phi) * phi
else:
    lo = np.floor(np.min(hm - 2 * hs) / phi) * phi
    hi = np.ceil(np.max(hm + 2 * hs) / phi) * phi
b = np.arange(lo - 0.5 * phi, hi + phi, phi)
c = np.zeros((len(b) - 1, gg.shape[1]))
for k in range(gg.shape[1]):
    c[:, k], _ = np.histogram(h[:, k], bins=b)
valid = c > 0
c[valid] = np.log10(c[valid] / r)
c[~valid] = np.nan
xx = np.arange(count + 1)
with plt.rc_context(context_properties):
    fig = plt.figure(dpi=110, figsize=(7.5, 4))
    ax = fig.add_axes((0.05, 0.05, 0.75, 0.9))
    ax.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
    ax.plot(xx, hm, linestyle='-', color=edgecolor)
    ax.plot(xx, hm - hs, linestyle='--', color=edgecolor, alpha=0.7)
    ax.plot(xx, hm + hs, linestyle='--', color=edgecolor, alpha=0.7)
    ax.plot([0, count + 0.5], [0, 0], linestyle=':', color=edgecolor, alpha = 0.7)
    msh = ax.pcolorfast(np.arange(-0.5, c.shape[1]), b, c, vmin=-2.2, vmax=0, cmap=cmap)
    cax = fig.add_axes((0.83, 0.3, 0.025, 0.65))
    cb = fig.colorbar(msh, cax=cax, ticks=np.log10([0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1]))
    cb.ax.set_yticklabels(['1%', '2%', '5%', '10%', '20%', '50%', '100%'])
    ax.set_xlabel('Days')
    ax.set_ylabel('Cummulative Gain (%)')
    ax.set_title('Ensemble Heat Map of Cummulative Gains')